In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import AbtBuyBenchmark

benchmark = AbtBuyBenchmark(data_dir_path="../data/")
benchmark

15:51:31 INFO:Extracting Abt-Buy...
15:51:31 INFO:Reading Abt-Buy record_dict...
15:51:31 INFO:Reading Abt-Buy train.csv...
15:51:31 INFO:Reading Abt-Buy valid.csv...
15:51:31 INFO:Reading Abt-Buy test.csv...


<AbtBuyBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Textual/Abt-Buy/abt_buy_exp_data.zip

## Preprocess

In [5]:
field_list = ['name', 'description', 'price']

In [6]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for record_dict in [benchmark.train_record_dict, benchmark.valid_record_dict, benchmark.test_record_dict]:
    for record in record_dict.values():
        for field in field_list:
            record[field] = clean_str(record[field])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
from entity_embed import PairNumericalizer

pair_numericalizer = PairNumericalizer(field_list)

In [9]:
batch_size = 32
eval_batch_size = 256
datamodule = benchmark.build_matcher_datamodule(
    pair_numericalizer=pair_numericalizer,
    batch_size=batch_size,
    eval_batch_size=eval_batch_size,
    random_seed=random_seed
)

## Training

In [10]:
from entity_embed import Matcher

model = Matcher(
    pair_numericalizer=pair_numericalizer
)

15:51:35 INFO:Load pretrained SentenceTransformer: stsb-distilbert-base
15:51:35 INFO:Did not find folder stsb-distilbert-base
15:51:35 INFO:Search model on server: http://sbert.net/models/stsb-distilbert-base.zip
15:51:35 INFO:Load SentenceTransformer from folder: /home/fjsj/.cache/torch/sentence_transformers/sbert.net_models_stsb-distilbert-base
15:51:36 INFO:Use pytorch device: cuda


In [11]:
trainer = model.fit(
    datamodule,
    min_epochs=5,
    max_epochs=100,
    check_val_every_n_epoch=1,
    early_stop_monitor="valid_f1_at_0.5",
    tb_save_dir='../tb_logs',
    tb_name=f'matcher-{benchmark.dataset_name}'
)

15:51:36 INFO:GPU available: True, used: True
15:51:36 INFO:TPU available: False, using: 0 TPU cores
15:51:36 INFO:Train positive pair count: 616
15:51:36 INFO:Train negative pair count: 5127
15:51:36 INFO:Valid positive pair count: 206
15:51:36 INFO:Valid positive pair count: 1710
15:51:36 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
15:51:41 INFO:
  | Name        | Type              | Params
--------------------------------------------------
0 | matcher_net | MatcherNet        | 66.4 M
1 | loss_fn     | BCEWithLogitsLoss | 0     
--------------------------------------------------
66.4 M    Trainable params
0         Non-trainable params
66.4 M    Total params
265.455   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 7.80 GiB total capacity; 2.99 GiB already allocated; 95.81 MiB free; 3.10 GiB reserved in total by PyTorch)

In [ ]:
trainer.fit(model, datamodule)

In [ ]:
model.validate(datamodule)

## Testing

In [ ]:
model.test(datamodule)